# IEEEG1PW

In [1]:
import matplotlib.pyplot as plt

import andes

In [2]:
%matplotlib inline

In [3]:
andes.config_logger(20)

In [4]:
!andes misc --version

Python  3.12.0
andes   1.9.3.post43+g99df4545
numpy   2.0.2
kvxopt  1.3.2.1
sympy   1.13.3
scipy   1.13.1
pandas  2.2.2
numba   0.60.0


In [5]:
s1 = andes.load(andes.get_case('kundur/kundur_ieeeg1.xlsx'),
                setup=False, no_output=True, default_config=True)
s2 = andes.load('./kundur_ieeeg1pw.xlsx',
                setup=False, no_output=True, default_config=True)

for ss in [s1, s2]:

    ss.Toggle.alter(src='u', idx=1, value=0)

    ss.add(model='Alter',
           param_dict=dict(t=1, model='PQ', dev='PQ_0', src='Ppf',
                           attr='v', method='+', amount=0.8))
    ss.add(model='Alter',
           param_dict=dict(t=1, model='PQ', dev='PQ_1', src='Ppf',
                           attr='v', method='+', amount=0.8))

    ss.setup()

for ss in [s1, s2]:
    # Constant load
    ss.PQ.config.p2p = 1
    ss.PQ.config.q2q = 1
    ss.PQ.config.p2z = 0
    ss.PQ.config.q2z = 0

    ss.PFlow.run()

Working directory: "/Users/jinningwang/work/demo/demo/IEEEG1PW"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "/Users/jinningwang/work/andes/andes/cases/kundur/kundur_ieeeg1.xlsx"...
Input file parsed in 0.3239 seconds.
Working directory: "/Users/jinningwang/work/demo/demo/IEEEG1PW"
> Reloaded generated Python code of module "pycode".
Parsing input file "./kundur_ieeeg1pw.xlsx"...
Input file parsed in 0.0207 seconds.
System internal structure set up in 0.0138 seconds.
System internal structure set up in 0.0131 seconds.
-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0020 seconds.
0: |F(x)| = 14.9282832
1: |F(x)| = 3.608627841
2: |F(x)| = 0.1701107882
3: |F(x)| = 0.002038626956
4: |F(x)| = 3.745103979e-07
Converged

In [6]:
for ss in [s1, s2]:
    _ = ss.TDS.init()

Initialization for dynamics completed in 0.0162 seconds.
Initialization was successful.
Suspect initialization issue! Simulation may crash!

      Name       | Var. Value | Eqn. Mismatch
-----------------+------------+--------------
 GV_y IEEEG1PW 1 | 7.268      | -7.268       


Initialization for dynamics completed in 0.0341 seconds.
Initialization failed!!
If you are developing a new model, check the initialization with
   andes -v 10 run -r tds --init ./kundur_ieeeg1pw.xlsx
Otherwise, check the variables that are initialized out of limits.


In [7]:
print('      PMIN:', s2.IEEEG1PW.PMIN.v[0])

for i in range(1, 7):
    print(f'Gv{i} * PMAX:', s2.IEEEG1PW.params[f'Gv{i}'].v[0] * s2.IEEEG1PW.PMAX.v[0])

print('      PMAX:', s2.IEEEG1PW.PMAX.v[0])

      PMIN: 0.0
Gv1 * PMAX: 3.8571428571428568
Gv2 * PMAX: 7.7142857142857135
Gv3 * PMAX: 11.571428571428571
Gv4 * PMAX: 15.428571428571427
Gv5 * PMAX: 19.285714285714285
Gv6 * PMAX: 23.142857142857142
      PMAX: 27.0


Below code is skipped for now

In [8]:
# for ss in [s1, s2]:
#     ss.TDS.config.tf = 300
#     ss.TDS.config.criteria = 0
#     ss.TDS.config.no_tqdm = True
#     # ss.TDS.run()

In [9]:
# plt.style.use('default')

# fig, ax = plt.subplots(2, 2, figsize=(15, 15))

# a=(3,)
# fmin, fmax = 58.5, 60.5
# _ = s1.TDS.plt.plot(s1.GENROU.omega,
#                     a=a,
#                     ymin=fmin, ymax=fmax,
#                     ytimes=s1.config.freq,
#                     grid=True, show=False, latex=False,
#                     yheader=['Case1: Linear'],
#                     title='a. Gen. Freq. [Hz]',
#                     fig=fig, ax=ax[0, 0])
# _ = s2.TDS.plt.plot(s2.GENROU.omega,
#                     a=a,
#                     ytimes=s1.config.freq,
#                     grid=True, show=False, latex=False,
#                     yheader=['Case2: Non-linear'],
#                     linestyles=['--'],
#                     fig=fig, ax=ax[0, 0])

# a= (0,)
# _ = s1.TDS.plt.plot(s1.IEEEG1.IAW_y,
#                     a=a,
#                     # ycalc=lambda x: x - s1.IEEEG1.v0.v[0],
#                     grid=True, show=False, latex=False,
#                     ymax=10,
#                     yheader=['Case1-Linear']*len(a),
#                     title='b. Valve position [p.u.]',
#                     fig=fig, ax=ax[0, 1])
# _ = s2.TDS.plt.plot(s2.IEEEG1NL.IAW_y,
#                     # ycalc=lambda x: x - s2.IEEEG1N.v0.v[0],
#                     grid=True, show=False, latex=False,
#                     yheader=['Case2-Non-linear']*len(a),
#                     linestyles=['--'],
#                     # ymin=5, ymax=15,
#                     fig=fig, ax=ax[0, 1])
# a= (0,)
# pmin, pmax = 7.2, 8.7
# _ = s1.TDS.plt.plot(s1.IEEEG1.GV,
#                     a=a,
#                     grid=True, show=False, latex=False,
#                     yheader=['Case1-Linear']*len(a),
#                     ymin=pmin, ymax=pmax,
#                     title='c. Steam flow [p.u.]',
#                     fig=fig, ax=ax[1, 0])
# _ = s2.TDS.plt.plot(s2.IEEEG1NL.GV,
#                     grid=True, show=False, latex=False,
#                     yheader=['Case2-Non-linear']*len(a),
#                     linestyles=['--'],
#                     # ymin=5, ymax=15,
#                     fig=fig, ax=ax[1, 0])

# _ = s1.TDS.plt.plot(s1.IEEEG1.pout,
#                     a=a,
#                     grid=True, show=False, latex=False,
#                     ymin=pmin, ymax=pmax,
#                     yheader=['Case1-Linear']*len(a),
#                     title='d. Mech. power [p.u.]',
#                     fig=fig, ax=ax[1, 1])
# _ = s2.TDS.plt.plot(s2.IEEEG1NL.pout,
#                     grid=True, show=False, latex=False,
#                     yheader=['Case2-Non-linear']*len(a),
#                     linestyles=['--'],
#                     fig=fig, ax=ax[1, 1])